In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Generate Dummy Data #

In [ ]:
X = np.linspace(0,3*np.pi,31)[:,None]
Y = np.sin(X)

# Setup of `GPy` #

In [ ]:
# GPy normalises Y, does this need to be done?

kern       = kern.RBF(X.shape[1])
likelihood = likelihoods.Gaussian(variance=noise_var)

inference_method = exact_gaussian_inference.ExactGaussianInference()

signal_variance = 1**2
lengthscale     = 1**2
noise_variance  = 1**2

# Parameters Changed #

In [ ]:
def _symmetrify_numpy(A, upper=False):
    triu = np.triu_indices_from(A,k=1)
    if upper:
        A.T[triu] = A[triu]
    else:
        A[triu] = A.T[triu]

def symmetrify(A, upper=False):
    """
    Take the square matrix A and make it symmetrical by copting elements from
    the lower half to the upper

    works IN PLACE.

    note: tries to use cython, falls back to a slower numpy version
    """
    #if config.getboolean('cython', 'working'):
    #    _symmetrify_cython(A, upper)
    #else:
        _symmetrify_numpy(A, upper)

def force_F_ordered(A):
    """
    return a F ordered version of A, assuming A is triangular
    """
    if A.flags['F_CONTIGUOUS']:
        return A
    print("why are your arrays not F order?")
    return np.asfortranarray(A)

def dpotri(A, lower=1):
    """
    Wrapper for lapack dpotri function

    DPOTRI - compute the inverse of a real symmetric positive
      definite matrix A using the Cholesky factorization A =
      U**T*U or A = L*L**T computed by DPOTRF

    :param A: Matrix A
    :param lower: is matrix lower (true) or upper (false)
    :returns: A inverse

    """
    if _fix_dpotri_scipy_bug:
        assert lower==1, "scipy linalg behaviour is very weird. please use lower, fortran ordered arrays"
        lower = 0

    A = force_F_ordered(A)
    R, info = lapack.dpotri(A, lower=lower) #needs to be zero here, seems to be a scipy bug

    symmetrify(R)
    return R, info

def dtrtri(L):
    """
    Inverts a Cholesky lower triangular matrix

    :param L: lower triangular matrix
    :rtype: inverse of L

    """

    L = force_F_ordered(L)
    return lapack.dtrtri(L, lower=1)[0]

def dpotrs(A, B, lower=1):
    """
    Wrapper for lapack dpotrs function
    :param A: Matrix A
    :param B: Matrix B
    :param lower: is matrix lower (true) or upper (false)
    :returns:
    """
    A = force_F_ordered(A)
    return lapack.dpotrs(A, B, lower=lower)

def jitchol(A, maxtries=5):
    A = np.ascontiguousarray(A)
    L, info = lapack.dpotrf(A, lower=1)
    if info == 0:
        return L
    else:
        diagA = np.diag(A)
        if np.any(diagA <= 0.):
            raise linalg.LinAlgError("not pd: non-positive diagonal elements")
        jitter = diagA.mean() * 1e-6
        num_tries = 1
        while num_tries <= maxtries and np.isfinite(jitter):
            try:
                L = linalg.cholesky(A + np.eye(A.shape[0]) * jitter, lower=True)
                return L
            except:
                jitter *= 10
            finally:
                num_tries += 1
        raise linalg.LinAlgError("not positive definite, even with jitter.")
    import traceback
    try: raise
    except:
        logging.warning('\n'.join(['Added jitter of {:.10e}'.format(jitter),
            '  in '+traceback.format_list(traceback.extract_stack(limit=2)[-2:-1])[0][2:]]))
    return L

def pdinv(A, *args):
    """
    :param A: A DxD pd numpy array

    :rval Ai: the inverse of A
    :rtype Ai: np.ndarray
    :rval L: the Cholesky decomposition of A
    :rtype L: np.ndarray
    :rval Li: the Cholesky decomposition of Ai
    :rtype Li: np.ndarray
    :rval logdet: the log of the determinant of A
    :rtype logdet: float64

    """
    L = jitchol(A, *args)
    logdet = 2.*np.sum(np.log(np.diag(L)))
    Li = dtrtri(L)
    Ai, _ = dpotri(L, lower=1)
    # Ai = np.tril(Ai) + np.tril(Ai,-1).T
    symmetrify(Ai)

    return Ai, L, Li, logdet

# GPy uses tdot_blas instead of np.dot (suggests it is good for "large" matricies)
def unscaled_dist(X, X2=None):
    """
    Compute the Euclidean distance between each row of X and X2, or between
    each pair of rows of X if X2 is None.
    """
    if X2 is None:
        Xsq = np.sum(np.square(X),1)
        r2 = -2.*np.dot(X, X.T) + (Xsq[:,None] + Xsq[None,:])
        util.diag.view(r2)[:,]= 0. # force diagnoal to be zero: sometime numerically a little negative
        r2 = np.clip(r2, 0, np.inf)
        return np.sqrt(r2)
    else:
        X1sq = np.sum(np.square(X),1)
        X2sq = np.sum(np.square(X2),1)
        r2 = -2.*np.dot(X, X2.T) + X1sq[:,None] + X2sq[None,:]
        r2 = np.clip(r2, 0, np.inf)

# self.posterior, self._log_marginal_likelihood, self.grad_dict = self.inference_method.inference(self.kern, self.X, 
#     self.likelihood, self.Y_normalized, self.mean_function, self.Y_metadata)
# m = 0
# YYT_factor = self.get_YYTfactor(Y-m)

YYT_factor = Y

# K = kern.K(X)
# r = self._scaled_dist(X, X2)
# r = self._unscaled_dist(X, X2)/self.lengthscale
r = unscaled_dist(X) / lengthscale:
# K = self.K_of_r(r)
K = signal_variance * np.exp(-0.5 * r**2)

Ky = K.copy()
diag.add(Ky, noise_variance+1e-8)
Wi, LW, LWi, W_logdet = pdinv(Ky)

alpha, _ = dpotrs(LW, YYT_factor, lower=1)

log_marginal =  0.5*(-Y.size * log_2_pi - Y.shape[1] * W_logdet - np.sum(alpha * YYT_factor))

dL_dK = 0.5 * (np.dot(alpha,alpha.T) - Y.shape[1] * Wi)

dL_dthetaL = likelihood.exact_inference_gradients(np.diag(dL_dK),Y_metadata)

return Posterior(woodbury_chol=LW, woodbury_vector=alpha, K=K), log_marginal, {'dL_dK':dL_dK, 'dL_dthetaL':dL_dthetaL, 'dL_dm':alpha}



# self.likelihood.update_gradients(self.grad_dict['dL_dthetaL'])
self.variance.gradient = grad

# self.kern.update_gradients_full(self.grad_dict['dL_dK'], self.X)
self.variance.gradient = np.sum(self.K(X, X2)* dL_dK)/self.variance
dL_dr = self.dK_dr_via_X(X, X2) * dL_dK
r = self._scaled_dist(X, X2)
self.lengthscale.gradient = -np.sum(dL_dr*r)/self.lengthscale

# Evaluate Likelihood and Derivatives #

In [ ]:
# call: m.log_likelihood()
self._log_marginal_likelihood
# call: m.gradient
self.gradient